In [ ]:
import pandas as pd
import json
import time
from selenium import webdriver
from browsermobproxy import Server
from selenium.webdriver.chrome.options import Options

In [ ]:
# Launch the proxy server, specify the full installation path:
# Запускаем proxy сервер, указываем полный путь установки:
server = Server('C:/Users/.../.../Jupyter Notebook/driver/\
                 browsermob-proxy-2.1.4/bin/browsermob-proxy')
server.start()
proxy = server.create_proxy()
# Configuring webdriver, including ignoring certificate errors:
# Настраиваем webdriver, в том числе на игнорирование ошибок сертификатов:
chrome_options = Options()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--proxy-server={0}'.format(proxy.proxy))
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
def catalog_list_url():
    """
    This function gets a complete list of links to the product catalog by
    capturing the json file of the directory from the XHR request passing
    through the proxy "localhost".
    ===========================================================================
    ===========================================================================
    Данная функция получает полный список ссылок на каталог товаров, захватывая
    json-файл каталога из XHR запроса, проходящего через proxy "localhost".
    """
    url = 'https://www.wildberries.ru'
    proxy.new_har(options={
        'captureContent': True,
        'captureHeaders': True
    })
    driver.get(url)
    time.sleep(15)
    # Getting the catalog:
    # Получаем каталог:
    catalog_result = proxy.har['log']['entries']
    catalog_list = []

    for catalog_record in catalog_result:
        if 'main-menu-ru-ru' in catalog_record['request']['url']:
            record_url = (
                json.loads(catalog_record['response']['content']['text'])
                )
            for record in record_url:
                try:
                    for child in record['childs']:
                        try:
                            if child['url']:
                                catalog_list.append(url + child['url'])
                            else:
                                continue
                        except:
                            continue
                        try:
                            for sub_child in child['childs']:
                                if sub_child['url']:
                                    catalog_list.append(url + sub_child['url'])
                                else:
                                    continue
                        except:
                            continue
                except:
                    continue
            return catalog_list

In [ ]:
def list_of_products():
    """
    This function obtains product data by capturing a json file from XHR
    request passing through the proxy "localhost", and adds them to the
    dataframe. As a consequence of the fact that Wildberries issues a maximum
    of 100 pages of products of one groups - in the range loop,
    we set the stop to 101.
    ==========================================================================
    ==========================================================================
    Данная функция получает данные о товарах захватывая json-файл из XHR
    запроса, проходящего через proxy "localhost", и добавляет их в датафрейм.
    В следсвие того, что Wildberries выдает максимум 100 страниц товаров одной
    группы - в цикле range выставляем стоп на 101.
    """
    products_list = []
    product_table = pd.DataFrame()
    for base_url in catalog_list_url():
        try:
            for i in range(1, 101):
                proxy.new_har(options={
                    'captureContent': True,
                    'captureHeaders': True
                    })
                driver.get(base_url + '?page=' + str(i))
                time.sleep(5)
                base_result = proxy.har['log']['entries']
                try:
                    for meaning in base_result:
                        if 'catalog?app' in meaning['request']['url'] \
                        and '&emp' not in meaning['request']['url']:
                            products_list.extend(
                                json.loads(
                                    meaning['response']['content']['text']
                                    )['data']['products']
                                )
                except:
                    continue
        except:
            continue
        # We go through each product and collect the data we are interested in:
        # Проходимся по каждому товару и собираем интересующие нас данные:
        for product in products_list:
            row = {'Артикул': product['id'], 'Бренд': product['brand'],
                   'Наименование товара': product['name'],
                   'Цена без скидки (RU)': int(product['priceU']) / 100,
                   'Скидка (%)': product['sale'],
                   'Цена со скидкой (RU)': int(product['salePriceU']) / 100}
            # Forming a dataframe:
            # Формируем датафрейм:
            product_table = (
                pd.concat(
                    [product_table, pd.DataFrame([row])]
                    )
                .drop_duplicates()
                )
    driver.quit()
    server.stop()
    return product_table.reset_index(drop=True)
list_of_products()